In [39]:
import numpy as np
from scipy.io import loadmat
import os
import pickle
import pandas as pd

In [33]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline


In [128]:
%%time
# load vgg

#vgg16_train = np.load('VGG16_pca_train.npy', allow_pickle=True)
#vgg16_val = np.load('VGG16_pca_val.npy', allow_pickle=True)
vgg16_label_train = np.load('VGG16_pca_label_train.npy', allow_pickle=True)
vgg16_label_val = np.load('VGG16_pca_label_val.npy', allow_pickle=True)

## load LGBPTOP
#
#LGBPTOP_train = np.load('LGBPTOP_pca_train.npy', allow_pickle=True)
#LGBPTOP_val = np.load('LGBPTOP_pca_val.npy', allow_pickle=True)
##LGBPTOP_label_train = np.load('VGG16_pca_label_train.npy', allow_pickle=True)
##LGBPTOP_label_val = np.load('VGG16_pca_label_val.npy', allow_pickle=True)
#
## load openface
#openface_train = np.load('itir_features_gazeR_AU_nomanual_train.npy', allow_pickle=True)
#openface_val = np.load('itir_features_gazeR_AU_nomanual_val.npy', allow_pickle=True)
#openface_colnames = np.load('itir_features_gazeR_AU_nomanual_columnames.npy', allow_pickle=True)



Wall time: 0 ns


In [130]:
# Baseline score 
dumb_prediction = 1 - mean_absolute_error(vgg16_label_val[:,1], np.full((2000,), np.mean(vgg16_label_train[:,1])))

dumb_prediction

0.8830107663551402

In [131]:
%%time

#n_list=[100, 300, 500, 1000,1500,2000]
n_list = [2000]

def run_pca_svr(train, val, train_label, val_label, model, component_list=list, stdscale = bool):
    if stdscale == True:
        scale = StandardScaler().fit(train)
        train = scale.transform(train)
        val = scale.transform(val)
    
    for n in n_list:    
        pca = PCA(n_components=n)
        pca.fit(train)
        print("explained variance ratio: ", pca.explained_variance_ratio_.sum())
        
        
        reduced_train = pca.transform(train)
        reduced_val = pca.transform(val)
        
        
        reg = model.fit(reduced_train, train_label[:,1])
        y_pred_lin = reg.predict(reduced_val)
        print('pca = ', n,' components')
        print('method =  svr')
        print(1 - mean_absolute_error(val_label[:,1], y_pred_lin))
        print(35*'-')
    
    return y_pred_lin
        



Wall time: 0 ns


In [132]:
%%time

###pca size
n_list = [2000]

###run singular pca-2000 on vgg
vgg16_train = np.load('VGG16_pca_train.npy', allow_pickle=True)
vgg16_val = np.load('VGG16_pca_val.npy', allow_pickle=True)
vgg16_preds = run_pca_svr(vgg16_train, vgg16_val, vgg16_label_train, vgg16_label_val, 
                           model=SVR(kernel='rbf'), component_list=n_list, stdscale = False)
del vgg16_train, vgg16_val


###run singular pca-2000 on lgbptop
LGBPTOP_train = np.load('LGBPTOP_pca_train.npy', allow_pickle=True)
LGBPTOP_val = np.load('LGBPTOP_pca_val.npy', allow_pickle=True)
LGBPTOP_preds = run_pca_svr(LGBPTOP_train, LGBPTOP_val, vgg16_label_train, vgg16_label_val, 
                           model=SVR(kernel='rbf'), component_list=n_list, stdscale = False)
del LGBPTOP_train, LGBPTOP_val




explained variance ratio:  0.9844880129961177
pca =  2000  components
method =  svr
0.9121766664501454
-----------------------------------
explained variance ratio:  0.9524866428030685
pca =  2000  components
method =  svr
0.9142625649748043
-----------------------------------
Wall time: 4min 47s


In [133]:
%%time

def lock_and_load(train_npy=str, val_npy=str, col_npy=str):
    train = np.load(train_npy, allow_pickle=True)
    val = np.load(val_npy, allow_pickle=True)
    colnames = np.load(col_npy, allow_pickle=True)
    
    df_train = pd.DataFrame(data=train,   # 1st column as index
             columns=colnames)  # 1st row as the column names
    df_val = pd.DataFrame(data=val,   # 1st column as index
             columns=colnames)  # 1st row as the column names
    
    return df_train, df_val


openface_df_train, openface_df_val = lock_and_load('itir_features_gazeR_AU_nomanual_train.npy',
                                 'itir_features_gazeR_AU_nomanual_val.npy',
                                'itir_features_gazeR_AU_nomanual_columnames.npy')

Wall time: 949 ms


In [134]:
openface_df_train.sort_values(by='video_id')

,video_id,gaze_angle_x_std,gaze_angle_y_std,pose_Rx_std,pose_Ry_std,pose_Rz_std,AU01_r_std,AU02_r_std,AU04_r_std,AU05_r_std,...,AU17_c_acceleration_skew,AU20_c_acceleration_skew,AU23_c_acceleration_skew,AU25_c_acceleration_skew,AU26_c_acceleration_skew,AU28_c_acceleration_skew,AU45_c_acceleration_skew,dist_i_acceleration_skew,interview_score_x,interview_score_y
4880,--Ymqszjv54.001,0.133361,0.0659834,0.0762868,0.152748,0.1414,0.269512,0.228604,0.0501309,0.197106,...,-0.329047,0,0,0,0,0,0,-0.194882,0.588785,0.588785
3840,--Ymqszjv54.003,0.174864,0.0760685,0.0910104,0.2299,0.127047,0.322073,0.295969,0.35671,0.271354,...,-0.471242,0,-0.125208,0,0,0,0,-1.12128,0.392523,0.392523
0,--Ymqszjv54.004,0.131222,0.134769,0.114069,0.166677,0.0912312,0.323816,0.562643,0.222843,0.607073,...,0.393559,0,0,0,0.316812,0,0,20.0986,0.401869,0.401869
960,--Ymqszjv54.005,0.166039,0.100606,0.0792788,0.182828,0.107713,0.568768,0.296307,0.409835,0.155511,...,0,0.780613,0,0.306081,0,0,0,-0.865409,0.280374,0.280374
2880,-2qsCrkXdWs.001,0.0973047,0.0638991,0.0649839,0.109546,0.0908634,0.236945,0.188475,0.327975,0.340441,...,-0.44092,0,0,0.0655232,-0.0500663,0,0,-0.804067,0.560748,0.560748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,zvX4fNj0uxk.004,0.123359,0.0549375,0.0645644,0.0993759,0.0315638,0.250367,0.234688,0.191126,0.117434,...,0.271902,0,0,-0.247714,0,0,-0.0797917,-0.329432,0.46729,0.46729
4854,zwO2zBdbCb4.002,0.16021,0.0548222,0.0568995,0.130871,0.0821942,0.546011,0.362547,0.181658,0.18327,...,0,2.01075,0,0,0,0,0,-1.17026,0.523364,0.523364
2871,zxuPCshTSOs.002,0.0659843,0.0731071,0.0626369,0.115254,0.0626409,0.786442,0.367898,0.563027,0.216554,...,0.179847,0.111206,0,0,0,0,0,-0.637549,0.420561,0.420561
3827,zxuPCshTSOs.005,0.0459678,0.0906235,0.0945196,0.0515769,0.038839,0.587535,0.533507,0.253088,0.145897,...,0,0,0,0,0,0,-0.125797,-0.874096,0.560748,0.560748


In [135]:
%%time
#  SVR

def calc_openface(df_train, df_val):
    #svr = make_pipeline(StandardScaler(), SVR(C=1, kernel='rbf', epsilon=0.01))
    svr = make_pipeline(StandardScaler(), SVR(kernel='rbf'))
    svr.fit(df_train.iloc[:,1:-2].values, df_train.iloc[:,-1].values)
    y_pred_svr = svr.predict(df_val.iloc[:,1:-2].values)
    score = 1 - mean_absolute_error(df_val.iloc[:,-1].values, y_pred_svr)
    print(score)
    return y_pred_svr
openface_itir_preds = calc_openface(openface_df_train, openface_df_val)


0.9010350783486331
Wall time: 11.5 s


# Now PCA VGG, LGBPTOP and concat openface features.

In [46]:
%%time
def load_lgbptop():
    data_dir = 'D:\\Thesis_data\\vgg_features'
    
    lgbptop_file_path = os.path.join(data_dir, 'lgbptop.mat')
    feats = loadmat(lgbptop_file_path)
    
    mdtype = feats['lgbptop'].dtype
    
    ndata = {n: feats['lgbptop'][n][0, 0] for n in mdtype.names}
    
    filenames = [filename[0] for filename in ndata['filename'].squeeze()]
    
    data = {'filenames': filenames, 'features': ndata['data']}

    return data

#lgbptop_data = load_lgbptop()

Wall time: 0 ns


In [136]:
# create vgg 2000 pca
def vgg16_pca():
    data_train = np.load('VGG16_pca_train.npy', allow_pickle=True)
    data_val = np.load('VGG16_pca_val.npy', allow_pickle=True)
    pca = PCA(n_components=2000)
    pca.fit(data_train)
    print("explained variance ratio: ", pca.explained_variance_ratio_.sum())
    
    
    reduced_train = pca.transform(data_train)
    reduced_val = pca.transform(data_val)
    
    return reduced_train, reduced_val


def lgbptop_pca():
    data_train = np.load('LGBPTOP_pca_train.npy', allow_pickle=True)
    data_val = np.load('LGBPTOP_pca_val.npy', allow_pickle=True)
    pca = PCA(n_components=2000)
    pca.fit(data_train)
    print("explained variance ratio: ", pca.explained_variance_ratio_.sum())
    
    
    reduced_train = pca.transform(data_train)
    reduced_val = pca.transform(data_val)
    
    return reduced_train, reduced_val


vgg16_pca_train, vgg16_pca_val = vgg16_pca()
lgbptop_pca_train, lgbptop_pca_val = lgbptop_pca()

#vgg16_label_train = np.load('VGG16_pca_label_train.npy', allow_pickle=True)
#vgg16_label_val = np.load('VGG16_pca_label_val.npy', allow_pickle=True)



explained variance ratio:  0.9991504425599264
explained variance ratio:  0.974207348186801


In [137]:
vgg16_label_train = np.load('VGG16_pca_label_train.npy', allow_pickle=True)
vgg16_label_val = np.load('VGG16_pca_label_val.npy', allow_pickle=True)

In [139]:
def calc_single(train, val, labels_train, labels_val):
    #svr = make_pipeline(StandardScaler(), SVR(C=1, kernel='rbf', epsilon=0.01))
    #svr = make_pipeline(StandardScaler(), SVR(kernel='rbf'))
    svr = SVR(kernel='rbf')
    svr.fit(train, labels_train)
    
    y_pred_svr = svr.predict(val)
    score = 1 - mean_absolute_error(labels_val, y_pred_svr)
    print(score)
    return y_pred_svr
calc_single(vgg16_pca_train, vgg16_pca_val, vgg16_label_train[:,1], vgg16_label_val[:,1])

0.9170658573550899


In [146]:
%%time
calc_single(np.hstack([vgg16_pca_train, lgbptop_pca_train]), 
            np.hstack([vgg16_pca_val, lgbptop_pca_val]), 
            vgg16_label_train[:,1], 
            vgg16_label_val[:,1])

0.9170765145422639
Wall time: 1min 6s


array([0.59421858, 0.4505491 , 0.48862649, ..., 0.44434176, 0.60268677,
       0.52755938])

In [151]:
pred_vgg = calc_single(vgg16_pca_train, vgg16_pca_val, vgg16_label_train[:,1], vgg16_label_val[:,1])
pred_lgbptop = calc_single(lgbptop_pca_train, lgbptop_pca_val, vgg16_label_train[:,1], vgg16_label_val[:,1])

0.9170658573550899
0.9145692497202029


In [152]:
(pred_vgg+pred_lgbptop)/2

array([0.63506513, 0.47348871, 0.50341167, ..., 0.44132775, 0.59949994,
       0.51900005])

In [154]:
1 - mean_absolute_error(vgg16_label_val[:,1], (pred_vgg+pred_lgbptop)/2)

0.9194789690229004

In [188]:
##### 3 models late fusion
#openface_itir_preds
1 - mean_absolute_error(vgg16_label_val[:,1], (pred_vgg+pred_lgbptop+openface_itir_preds)/3)

0.9194892389622538

In [148]:
#deepmodel_label_train = np.load('VGG16_pca_label_train.npy', allow_pickle=True)
#deepmodel_label_val = np.load('VGG16_pca_label_val.npy', allow_pickle=True)
#
#openface_df_train = openface_df_train.sort_values(by='video_id')
#openface_df_val = openface_df_val.sort_values(by='video_id')
#
#vgg_train = pd.concat([pd.DataFrame(deepmodel_label_train[:,0]), pd.DataFrame(vgg16_pca_train),pd.DataFrame(deepmodel_label_train[:,1])],axis=1)
#vgg_val = pd.concat([pd.DataFrame(deepmodel_label_val[:,0]), pd.DataFrame(vgg16_pca_val), pd.DataFrame(deepmodel_label_val[:,1])],axis=1)
#vgg_train = vgg_train.T.reset_index(drop=True).T
#vgg_val = vgg_val.T.reset_index(drop=True).T
#vgg_train = vgg_train.sort_values(by=vgg_train.columns[0], ascending = True)
#vgg_val = vgg_val.sort_values(by=vgg_val.columns[0], ascending = True)
#
#lgbptop_train = pd.concat([pd.DataFrame(deepmodel_label_train[:,0]), pd.DataFrame(lgbptop_pca_train),pd.DataFrame(deepmodel_label_train[:,1])],axis=1)
#lgbptop_val = pd.concat([pd.DataFrame(deepmodel_label_train[:,0]), pd.DataFrame(lgbptop_pca_val),pd.DataFrame(deepmodel_label_train[:,1])],axis=1)
#lgbptop_train = vgg_train.T.reset_index(drop=True).T
#lgbptop_val = vgg_val.T.reset_index(drop=True).T
#lgbptop_train = lgbptop_train.sort_values(by=lgbptop_train.columns[0], ascending = True)
#lgbptop_val = vgg_val.sort_values(by=vgg_val.columns[0], ascending = True)

In [161]:
##print(vgg16_label_train[:,0])
#vgg16_label_train[vgg16_label_train[:,0].argsort()]

array([['--Ymqszjv54.001', 0.5887850467289719],
       ['--Ymqszjv54.003', 0.3925233644859813],
       ['--Ymqszjv54.004', 0.40186915887850466],
       ...,
       ['zxuPCshTSOs.002', 0.4205607476635514],
       ['zxuPCshTSOs.005', 0.5607476635514018],
       ['zyGz_H1UTnQ.003', 0.4859813084112149]], dtype=object)

In [157]:
#openface_df_train

,video_id,gaze_angle_x_std,gaze_angle_y_std,pose_Rx_std,pose_Ry_std,pose_Rz_std,AU01_r_std,AU02_r_std,AU04_r_std,AU05_r_std,...,AU17_c_acceleration_skew,AU20_c_acceleration_skew,AU23_c_acceleration_skew,AU25_c_acceleration_skew,AU26_c_acceleration_skew,AU28_c_acceleration_skew,AU45_c_acceleration_skew,dist_i_acceleration_skew,interview_score_x,interview_score_y
4880,--Ymqszjv54.001,0.133361,0.0659834,0.0762868,0.152748,0.1414,0.269512,0.228604,0.0501309,0.197106,...,-0.329047,0,0,0,0,0,0,-0.194882,0.588785,0.588785
3840,--Ymqszjv54.003,0.174864,0.0760685,0.0910104,0.2299,0.127047,0.322073,0.295969,0.35671,0.271354,...,-0.471242,0,-0.125208,0,0,0,0,-1.12128,0.392523,0.392523
0,--Ymqszjv54.004,0.131222,0.134769,0.114069,0.166677,0.0912312,0.323816,0.562643,0.222843,0.607073,...,0.393559,0,0,0,0.316812,0,0,20.0986,0.401869,0.401869
960,--Ymqszjv54.005,0.166039,0.100606,0.0792788,0.182828,0.107713,0.568768,0.296307,0.409835,0.155511,...,0,0.780613,0,0.306081,0,0,0,-0.865409,0.280374,0.280374
2880,-2qsCrkXdWs.001,0.0973047,0.0638991,0.0649839,0.109546,0.0908634,0.236945,0.188475,0.327975,0.340441,...,-0.44092,0,0,0.0655232,-0.0500663,0,0,-0.804067,0.560748,0.560748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,zvX4fNj0uxk.004,0.123359,0.0549375,0.0645644,0.0993759,0.0315638,0.250367,0.234688,0.191126,0.117434,...,0.271902,0,0,-0.247714,0,0,-0.0797917,-0.329432,0.46729,0.46729
4854,zwO2zBdbCb4.002,0.16021,0.0548222,0.0568995,0.130871,0.0821942,0.546011,0.362547,0.181658,0.18327,...,0,2.01075,0,0,0,0,0,-1.17026,0.523364,0.523364
2871,zxuPCshTSOs.002,0.0659843,0.0731071,0.0626369,0.115254,0.0626409,0.786442,0.367898,0.563027,0.216554,...,0.179847,0.111206,0,0,0,0,0,-0.637549,0.420561,0.420561
3827,zxuPCshTSOs.005,0.0459678,0.0906235,0.0945196,0.0515769,0.038839,0.587535,0.533507,0.253088,0.145897,...,0,0,0,0,0,0,-0.125797,-0.874096,0.560748,0.560748


In [187]:
#%%time
##test stacking all visual features
#temp_train = np.hstack([vgg16_label_train, vgg16_pca_train, lgbptop_pca_train])
#temp_train = temp_train[temp_train[:,0].argsort()]
#temp_train = np.hstack([temp_train, openface_df_train.iloc[:,1:-2]])
#
#
#temp_val = np.hstack([vgg16_label_val, vgg16_pca_val, lgbptop_pca_val])
#temp_val = temp_val[temp_val[:,0].argsort()]
#temp_val = np.hstack([temp_val, openface_df_val.iloc[:,1:-2]])
#
#sorted_labels_train = vgg16_label_train[vgg16_label_train[:,0].argsort()]
#sorted_labels_val = vgg16_label_val[vgg16_label_val[:,0].argsort()]
#
#calc_single(temp_train[:,2:], 
#            temp_val[:,2:], 
#            sorted_labels_train[:,1], 
#            sorted_labels_val[:,1])

0.9110307860970858
Wall time: 1min 33s


array([0.61869107, 0.46490458, 0.54842637, ..., 0.4320147 , 0.43059838,
       0.41874782])

In [149]:
#%%time
##  SVR single from pandas array. DOING THIS SHIT FROM PANDAS ARRAY SUCKS ASS. VERY POOR PERFORMANCE
#
#def calc(df_train, df_val):
#    svr = SVR(kernel='rbf')
#    svr.fit(df_train.iloc[:,1:-1].values, df_train.iloc[:,-1].values)
#    y_pred_svr = svr.predict(df_val.iloc[:,1:-1].values)
#    score = 1 - mean_absolute_error(df_val.iloc[:,-1].values, y_pred_svr)
#    print(score)
#    return y_pred_svr
#calc_openface(vgg_train, vgg_val)

0.905207956852081
Wall time: 1min 7s


array([0.59458447, 0.50177566, 0.50849362, ..., 0.47456428, 0.49479596,
       0.48309026])

# Sound analysis

In [189]:
def wav_to_features(results_folder=str):
    path_base = results_folder
    
    file_locations = []
    video_id = []
    
    for file in os.listdir(path_base):
        if file.endswith(".csv"):
            file_locations.append(os.path.join(path_base, file))
            video_id.append(file)
            
    file_locations.sort()
    video_id.sort()
            
    return file_locations, video_id


In [206]:
%%time 

def make_featureset(file_locations=list):
    # find necessary dimension size
    path = file_locations[0]
    f=open(path, "r")
    ### Go to the last line
    last_line=f.readlines()[-1]
    f.close()
    feature_size=last_line.split(',')
     ### Remove the first and last element
    feature_size=np.array(feature_size[1:-1])
    feature_size = feature_size.shape[0]
    
    ### loop over files to create dataset
    final_dataset = np.zeros((len(file_locations), feature_size))
    for i in range(len(file_locations)):
        path = file_locations[i]
        
        f = open(path,"r")
        ### Go to the last line. Last line contains feature values
        last_line=f.readlines()[-1]
        f.close()
        features=last_line.split(',')
        
        ### Remove the first and last element. First element = "unknown", last element = "?"
        features=np.array(features[1:-1])
        
        final_dataset[i, :] = features
        
        if i%400 == 0: # measures progress
            print(i) 

    return final_dataset

Wall time: 0 ns


In [207]:
%%time
#  SVR single from pandas array. DOING THIS SHIT FROM PANDAS ARRAY SUCKS ASS. VERY POOR PERFORMANCE

def calc(train, val, sorted_labels_train, sorted_labels_val):
    svr = SVR(kernel='rbf')
    #svr = make_pipeline(StandardScaler(), SVR(kernel='rbf'))
    svr.fit(train, sorted_labels_train)
    y_pred_svr = svr.predict(val)
    score = 1 - mean_absolute_error(sorted_labels_val, y_pred_svr)
    print(score)
    return y_pred_svr



Wall time: 0 ns


In [215]:
%%time

emo_large_file_locations_train, emo_large_csv_names_train = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_emo_large_train')
emo_large_file_locations_val, emo_large_csv_names_val = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_emo_large_val')
gemaps_file_locations_train, gemaps_csv_names_train = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_gemaps_train')
gemaps_file_locations_val, gemaps_csv_names_val = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_gemaps_val')
IS_09_file_locations_train, IS_09_csv_names_train = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_IS09_emotion_train')
IS_09_file_locations_val, IS_09_csv_names_val = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_IS09_emotion_val')
IS_10_file_locations_train, IS_10_csv_names_train = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_IS10_paraling_train')
IS_10_file_locations_val, IS_10_csv_names_val = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_IS10_paraling_val')
IS_11_file_locations_train, IS_11_csv_names_train = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_IS11_speaker_state_train')
IS_11_file_locations_val, IS_11_csv_names_val = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_IS11_speaker_state_val')

#IS_13_file_locations_train, IS_13_csv_names_train = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_IS13_ComParE_train')
#IS_13_file_locations_val, IS_13_csv_names_val = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_IS13_ComParE_val')

all_sets_train = [emo_large_file_locations_train, gemaps_file_locations_train,
                 IS_09_file_locations_train, IS_10_file_locations_train,
                 IS_11_file_locations_train]
all_sets_val = [emo_large_file_locations_val, gemaps_file_locations_val,
                 IS_09_file_locations_val, IS_10_file_locations_val,
                 IS_11_file_locations_val]

def loop_sound_data():
    predictions_sound = []
    for i in range(len(all_sets_train)):
        train = make_featureset(all_sets_train[i])
        val = make_featureset(all_sets_val[i])
        preds = calc_single(train, val, sorted_labels_train[:,1], sorted_labels_val[:,1])
        predictions_sound.append(preds)
    return predictions_sound

preds_all = loop_sound_data()

#IS_11_train = make_featureset(IS_11_file_locations_train)
#IS_11_val = make_featureset(IS_11_file_locations_val)
#calc(IS_11_train, IS_11_val, sorted_labels_train[:,1], sorted_labels_val[:,1])
#del IS_11_train, IS_11_val
#
#emo_large_train = make_featureset(emo_large_file_locations_train)
#emo_large_val = make_featureset(emo_large_file_locations_val)
#calc(emo_large_train, emo_large_val, sorted_labels_train[:,1], sorted_labels_val[:,1])
#del emo_large_train, emo_large_val
#
#gemaps_train = make_featureset(gemaps_file_locations_train)
#gemaps_val = make_featureset(gemaps_file_locations_val)
#calc(gemaps_train, gemaps_val, sorted_labels_train[:,1], sorted_labels_val[:,1])
#
#del gemaps_train, gemaps_val

0
400
800
1200
1600
2000
2400
2800
3200
3600
4000
4400
4800
5200
5600
0
400
800
1200
1600
0.879552250799446
0
400
800
1200
1600
2000
2400
2800
3200
3600
4000
4400
4800
5200
5600
0
400
800
1200
1600
0.8816209249862806
0
400
800
1200
1600
2000
2400
2800
3200
3600
4000
4400
4800
5200
5600
0
400
800
1200
1600
0.8802133857655539
0
400
800
1200
1600
2000
2400
2800
3200
3600
4000
4400
4800
5200
5600
0
400
800
1200
1600
0.8742469227600814
0
400
800
1200
1600
2000
2400
2800
3200
3600
4000
4400
4800
5200
5600
0
400
800
1200
1600
0.8811368644950229
Wall time: 14min 48s


In [228]:
#np.vstack([preds_all[0], preds_all[1], preds_all[2], preds_all[3], preds_all[4]]).shape
1 - mean_absolute_error(sorted_labels_val[:,1], ((preds_all[0] + preds_all[1] + preds_all[2] + preds_all[3] + preds_all[4])/5))


0.882199418251305

In [ ]:
%%time

emo_large_file_locations_train, emo_large_csv_names_train = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_emo_large_train')
emo_large_file_locations_val, emo_large_csv_names_val = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_emo_large_val')
gemaps_file_locations_train, gemaps_csv_names_train = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_gemaps_train')
gemaps_file_locations_val, gemaps_csv_names_val = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_gemaps_val')
IS_09_file_locations_train, IS_09_csv_names_train = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_IS09_emotion_train')
IS_09_file_locations_val, IS_09_csv_names_val = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_IS09_emotion_val')
IS_10_file_locations_train, IS_10_csv_names_train = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_IS10_paraling_train')
IS_10_file_locations_val, IS_10_csv_names_val = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_IS10_paraling_val')
IS_11_file_locations_train, IS_11_csv_names_train = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_IS11_speaker_state_train')
IS_11_file_locations_val, IS_11_csv_names_val = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_IS11_speaker_state_val')

#IS_13_file_locations_train, IS_13_csv_names_train = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_IS13_ComParE_train')
#IS_13_file_locations_val, IS_13_csv_names_val = wav_to_features('D:\Thesis_2020\opensmile-3.0-win-x64\RESULTS_IS13_ComParE_val')

all_sets_train = [emo_large_file_locations_train, gemaps_file_locations_train,
                 IS_09_file_locations_train, IS_10_file_locations_train,
                 IS_11_file_locations_train]
all_sets_val = [emo_large_file_locations_val, gemaps_file_locations_val,
                 IS_09_file_locations_val, IS_10_file_locations_val,
                 IS_11_file_locations_val]

def make_pca_sound_data():
    pca_sound = []
    for i in range(len(all_sets_train)):
        train = make_featureset(all_sets_train[i])
        val = make_featureset(all_sets_val[i])
        preds = calc_single(train, val, sorted_labels_train[:,1], sorted_labels_val[:,1])
        pca_sound.append(preds)
    return pca_sound

preds_all = loop_sound_data()



In [233]:
def pca(data_train, data_val):
    pca = PCA(n_components=500)
    pca.fit(data_train)
    print("explained variance ratio: ", pca.explained_variance_ratio_.sum())
    
    
    reduced_train = pca.transform(data_train)
    reduced_val = pca.transform(data_val)
    
    return reduced_train, reduced_val

pca(make_featureset(all_sets_train[0]), make_featureset(all_sets_val[0]))

0
400
800
1200
1600
2000
2400
2800
3200
3600
4000
4400
4800
5200
5600
0
400
800
1200
1600
explained variance ratio:  0.9999999999999998


(array([[ 1.63553780e+15, -4.20057778e+14, -3.52826008e+14, ...,
          2.02567090e+04, -1.50112026e+05,  2.01890546e+04],
        [ 1.91025239e+15, -6.19034680e+14, -6.02771242e+14, ...,
         -4.54940958e+04,  2.16749782e+04, -8.74858315e+04],
        [ 1.05815182e+15, -8.38639103e+13,  4.19433319e+13, ...,
          7.77675244e+04, -5.10134600e+04, -5.22886228e+04],
        ...,
        [-1.97687559e+14, -5.78054480e+13,  2.66713721e+13, ...,
         -2.85762278e+04, -9.72400282e+03,  1.68407020e+04],
        [-1.21065561e+14, -1.17669254e+14, -1.03579083e+13, ...,
         -6.32544204e+03, -2.65551443e+04, -3.90245892e+03],
        [-2.96466477e+14, -4.87005882e+13,  2.01864921e+13, ...,
          3.72952053e+03,  2.49400974e+04,  2.92180970e+03]]),
 array([[ 1.05815182e+15, -8.38639103e+13,  4.19433319e+13, ...,
          7.77675244e+04, -5.10134600e+04, -5.22886228e+04],
        [ 1.50871946e+14,  5.18285434e+14, -4.55730428e+14, ...,
          6.43635438e+04,  1.35788815e

# Textual

In [ ]:
# create 2 lists: (1) file locations (2) video id's
#path_base = "D:\\Thesis_2020\\OpenFace_2.2.0_win_x64\\processed - train - "
path_base = "D:\\Thesis_data\\train-"

# multi path, iterate over folders
folder_numbers = range(1,7)
file_locations = []
video_id = []

for i in folder_numbers:
    path = path_base + str(i)
    print(path)
    for file in os.listdir(path):
        if file.endswith(".mp4"):
            file_locations.append(os.path.join(path, file))
            video_id.append(file)

file_locations_train, mp4_names_train = file_locations, video_id

path_base = "D:\\Thesis_data\\val-"

# multi path, iterate over folders
folder_numbers = range(1,3)
file_locations = []
video_id = []

for i in folder_numbers:
    path = path_base + str(i)
    print(path)
    for file in os.listdir(path):
        if file.endswith(".mp4"):
            file_locations.append(os.path.join(path, file))
            video_id.append(file)
            
file_locations_val, mp4_names_val = file_locations, video_id

In [ ]:
transcription_train = np.load('transcription_training.pkl', allow_pickle=True)
transcription_val = np.load('transcription_validation.pkl', allow_pickle=True)

train_keys = np.array(list(transcription_train.keys()))
train_texts = np.array(list(transcription_train.values()))
text_train = np.vstack((train_keys, train_texts)).T

#first column contains video name, second column contains transcription

val_keys = np.array(list(transcription_val.keys()))
val_texts = np.array(list(transcription_val.values()))
text_val = np.vstack((val_keys, val_texts)).T

#first column contains video name, second column contains transcription



In [ ]:
# sort train and label data
text_train = np.sort(text_train, axis=0)
text_val = np.sort(text_val, axis=0)
train_labels = np.sort(train_labels, axis=0)
val_labels = np.sort(val_labels, axis=0)

print(text_train[5550, 0])
print(train_labels[5550, 0])

print(text_val[1333, 0])
print(val_labels[1333, 0])

# Openface exploration

In [ ]:
## correlations

corr = df_train.iloc[:,1:].astype(str).astype(float).corr()
def inspect_corr(partial_feature_name=str):
    series = (corr.iloc[-1, corr.columns.str.contains(partial_feature_name)]).sort_values()
    
    return series

inspect_corr(partial_feature_name='R')